In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
import numpy as np 
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from matplotlib import pyplot as plt

In [5]:
pd.set_option('display.max_columns', 400)
pd.set_option('display.max_rows', 30)

In [6]:
col_names = ['name', 'views_num', 'area', 'COORD_X', 'COORD_Y',
             'city', 'address', 'district', 'underground', 'rooms', 
             'floor', 'square', 'height', 'build_year', 'toilet',
             'balcony', 'pol', 'repair', 'price', 'terms_sale', 
             'property', 'shop', 'pharmacy', 'updated', 'agency', 'video']
col_names = [i.upper() for i in col_names]
col_names

df = pd.read_csv('realtby_end_3.0.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [8]:
class ViewsEncoder(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        values = X['VIEWS_NUM']
        values = [re.findall(r'\d+', i)[0::2] if not pd.isna(i) else [] for i in values]
        for i in range(len(values)):
            for j in range(2 - len(values[i])):
                values[i].append(np.nan)
        nums = np.array([[float(j) for j in i] for i in values])
        X['VIEWS_TODAY'], X['VIEWS_LAST_WEEK'] = nums[:, 0], nums[:, 1]
        X.drop(['VIEWS_NUM'], axis=1, inplace=True)
        return X

In [9]:
ven = ViewsEncoder()
df = ven.fit_transform(df)

In [10]:
class NumericFilter(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        for i in ['SHOP', 'PHARMACY', 'UNDERGROUND']:
            X[i] = X[i].map(self.preprocessing_handler_int)
        for i in ['HEIGHT']:
            X[i] = X[i].map(self.preprocessing_handler_float)
            
        values = X['ROOMS']
        values = [re.findall(r"\d+", i) if not pd.isna(i) else [] for i in values]
        for i in range(len(values)):
            for j in range(1 - len(values[i])):
                values[i].append(np.nan)
        values = [float(i[0]) for i in values]
        values = np.array(values)
        X['ROOMS'] = values
        
        values = X['FLOOR']
        values = [re.findall(r"[-+]?\d*\.\d+|\d+", i) if not pd.isna(i) else [] for i in values]
        for i in range(len(values)):
            for j in range(2 - len(values[i])):
                values[i].append(np.nan)
        values = [[float(j) for j in i] for i in values]
        values = np.array(values)
        X['FLOOR_NOW'] = values[:, 0]
        X['FLOOR_MAX'] = values[:, 1]
        X.drop(['FLOOR'], axis=1, inplace=True)
        
        values = X['SQUARE']
        values = [re.findall(r"[-+]?\d*\.\d+|\d+", i) if not pd.isna(i) else [] for i in values]
        for i in range(len(values)):
            for j in range(3 - len(values[i])):
                values[i].append(np.nan)
        values = [[float(j) for j in i] for i in values]
        values = np.array(values)
        X['SQUARE_SUM'] = values[:, 0]
        X['SQUARE_LIVE'] = values[:, 1]
        X['SQUAER_KITCHEN'] = values[:, 2]
        X.drop(['SQUARE'], axis=1, inplace=True)
        
        return X
    
    def preprocessing_handler_int(self, x):
        if pd.isna(x):
            return x
        items = re.findall(r'\d+', x)
        if len(items) < 1:
            return np.nan
        else:
            return int(items[0])
        
    def preprocessing_handler_float(self, x):
        if pd.isna(x):
            return x
        items = re.findall(r"[-+]?\d*\.\d+|\d+|\d*\,\d+", x)
        if len(items) < 1:
            return np.nan
        else:
            return float(items[0])

In [11]:
nf = NumericFilter()
df = nf.fit_transform(df)

[ 1.  1. 47. ...  9.  9. nan]


In [12]:
df

,NAME,AREA,COORD_X,COORD_Y,CITY,ADDRESS,DISTRICT,UNDERGROUND,ROOMS,HEIGHT,BUILD_YEAR,TOILET,BALCONY,POL,REPAIR,PRICE,TERMS_SALE,PROPERTY,SHOP,PHARMACY,UPDATED,AGENCY,VIDEO,FULL,VIEWS_TODAY,VIEWS_LAST_WEEK,FLOOR_NOW,FLOOR_MAX,SQUARE_SUM,SQUARE_LIVE,SQUAER_KITCHEN
0,Продажа 1/2 доли в 3-комнатной квартире в г. Г...,Гродненская область,30.247803,53.927870,г. Гродно,NaN,NaN,NaN,1.0,2.52,1980.0,раздельный,балкон и лоджия,деревянные,без отделки,"21 526 руб, 1 076 руб/кв.м.",чистая продажа,гос.-приватизированная,200.0,500.0,2019-09-25,NaN,Фотографии и планировки объекта:;;,Ленинский район: Пересёлка-4,3.0,81.0,6.0,9.0,57.0,20.0,7.0
1,Продажа 1/2 доли в 3-комнатной квартире в г. Г...,Гродненская область,23.816521,53.697233,г. Гродно,NaN,NaN,NaN,1.0,2.52,1980.0,раздельный,балкон и лоджия,деревянные,без отделки,"21 526 руб, 1 076 руб/кв.м.",чистая продажа,гос.-приватизированная,200.0,500.0,2019-09-25,NaN,null;;,Ленинский район: Пересёлка-4,4.0,82.0,6.0,9.0,57.0,20.0,7.0
2,Продажа комнаты в квартире в Кобринском районе...,Брестская область,30.247803,53.927870,г. Кобрин,NaN,NaN,NaN,47.0,2.50,1979.0,раздельный,лоджия,NaN,плохое состояние,"22 551 руб, 1 611 руб/кв.м.",чистая продажа,частная,200.0,200.0,2019-08-23,АЛЬТЕРНАТИВА Брест,Фотографии и планировки объекта:;;,NaN,6.0,69.0,5.0,5.0,49.1,14.0,7.2
3,Продажа комнаты в квартире в Кобринском районе...,Брестская область,24.353911,52.208929,г. Кобрин,NaN,NaN,NaN,47.0,2.50,1979.0,раздельный,лоджия,NaN,плохое состояние,"22 551 руб, 1 611 руб/кв.м.",чистая продажа,частная,200.0,200.0,2019-08-23,АЛЬТЕРНАТИВА Брест,null;;,NaN,8.0,71.0,5.0,5.0,49.1,14.0,7.2
4,Продажа доли в 3-х комнатной квартире,Минская область,30.247803,53.927870,г. Марьина Горка,"Новая заря ул., 2",NaN,NaN,1.0,NaN,1949.0,раздельный,лоджия,деревянные,нормальный ремонт,"22 551 руб, 469 руб/кв.м.",чистая продажа,NaN,350.0,700.0,2019-11-21,Квадратный метр,Фотографии и планировки объекта:;;,NaN,12.0,114.0,2.0,4.0,71.4,48.1,9.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12782,"Продается квартира 8 комнат д. Тарасово, ул. Р...",Минская область,27.391663,53.915300,д. Тарасово,"Ратомская ул., 1",NaN,NaN,8.0,2.64,2012,2 сан.узла,лоджия,деревянные,отличный ремонт,"861 042 руб, 3 381 руб/кв.м.",чистая продажа,частная,NaN,NaN,2019-11-22,Агентство недвижимости Особый стиль,null;;;;,NaN,3.0,33.0,2.0,NaN,254.7,151.8,61.3
12783,Продается современная 2-х этажная квартира пен...,Минская область,27.677200,53.946800,г. Минск,"Городецкая ул., 32 Информация о доме",Первомайский район,700.0,8.0,NaN,2000,3 сан.узла,3 балкона,паркет,евроремонт,"1,17 млн руб, 3 840 руб/кв.м.",чистая продажа,частная,100.0,200.0,2019-11-22,Тэтравита,null;;;;,Первомайский район: Уручье,12.0,95.0,7.0,9.0,304.3,211.3,19.8
12784,"Продажа 9-и комнатной квартиры, г. Минск, ул. ...",Минская область,27.678500,53.883600,г. Минск,"Малое Стиклево ул., 5 Информация о доме",Заводской район,NaN,9.0,3.20,2010,3 сан.узла,нет,ламинированные,нормальный ремонт,"389 519 руб, 1 267 руб/кв.м.",чистая продажа,частная,300.0,200.0,2019-09-26,Эксперт,null;;;;,Заводской район: Ангарская,15.0,61.0,3.0,NaN,307.4,191.3,NaN
12785,Продаётся уникальная 9к квартира в центре Минска.,Минская область,27.526400,53.913900,г. Минск,"Москвина ул., 8 Информация о доме",Центральный район,900.0,9.0,2.98,2003,3 сан.узла,2 балкона,паркет,хороший ремонт,"861 042 руб, 1 833 руб/кв.м.",чистая продажа,частная,200.0,350.0,NaN,NaN,null;;;;,"Центральный район: Победителей, Заславская, Г...",13.0,111.0,4.0,5.0,470.0,349.0,27.1


In [13]:
class PriceTransformer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['PRICE'] = X['PRICE'].map(self.price_prep)
        X = X[~X['PRICE'].isna()]
        return X
        
    def price_prep(self, s):   
        if pd.isna(s):
            return float('nan')
        if s == 'Цена договорная':
            return float('nan')

        if s.find('млн') != -1:        
            s = s[:s.find('млн')].replace('\xa0', '')        
            return int(float(s.replace(',', '.'))*1000000)

        return int(s.split(',')[0].replace('руб', '').replace('\xa0', ''))

    

In [14]:
ptr = PriceTransformer()
df = ptr.fit_transform(df)

In [15]:
class DateTransformer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['DAYS'] = (pd.to_datetime('2019-11-24') - pd.to_datetime(X['UPDATED'])).dt.days
        X.drop(['UPDATED'], axis=1, inplace=True)
        return X

In [16]:
dtr = DateTransformer()
df = dtr.fit_transform(df)

In [17]:
class ToBinaryTransformer(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X['TERMS_SALE'] = X['TERMS_SALE'].map(lambda x: 'nan' if pd.isna(x) else x)
        X['TERMS_SALE'] = X['TERMS_SALE'].map(lambda x: 1 if x.strip() == 'чистая продажа' else 0)
        X['LOC_TYPE'] = X.CITY.str.split().str[0].apply(lambda x: None if '.' not in str(x) else x)
        return X

In [18]:
to_bin = ToBinaryTransformer()
df = to_bin.fit_transform(df)

In [19]:
class CategoricalTransform(BaseEstimator, TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X.loc[X.AGENCY.notna(), 'AGENCY'] = 1
        X.loc[X.AGENCY.isna(), 'AGENCY'] = 0

        X.loc[X.PROPERTY != 'частная', 'PROPERTY'] = 0
        X.loc[X.PROPERTY == 'частная', 'PROPERTY'] = 1

        X.loc[X.REPAIR == 'евроремонт', 'REPAIR'] = 3
        X.loc[X.REPAIR == 'отличный ремонт', 'REPAIR'] = 3
        X.loc[X.REPAIR == 'хороший ремонт', 'REPAIR'] = 2
        X.loc[X.REPAIR == 'нормальный ремонт', 'REPAIR'] = 1
        X.loc[X.REPAIR == 'удовлетворительный ремонт', 'REPAIR'] = 1
        X.loc[X.REPAIR == 'строительная отделка', 'REPAIR'] = 0
        X.loc[X.REPAIR == 'без отделки', 'REPAIR'] = 0
        X.loc[X.REPAIR == 'аварийное состояние', 'REPAIR'] = 0
        X.loc[X.REPAIR == 'плохое состояние', 'REPAIR'] = 0
        
        return X

In [20]:
catr = CategoricalTransform()
df = catr.fit_transform(df)

In [21]:
df.drop(['NAME', 'ADDRESS', 'PROPERTY', 'VIDEO', 'POL'], axis=1, inplace=True)

In [26]:
df['COEFF'] = df['VIEWS_LAST_WEEK'] / df['DAYS']
inp_coeff = SimpleImputer(strategy='mean', missing_values=np.nan)
df[['COEFF']] = inp_coeff.fit_transform(df[['COEFF']])
mm_sc = MinMaxScaler()
df[['COEFF']] = mm_sc.fit_transform(df[['COEFF']])

In [28]:
num_features_max = [
    'UNDERGROUND',
    'SHOP', 'PHARMACY'
]
num_features_mean = [
    'COORD_X', 'COORD_Y', 'VIEWS_LAST_WEEK', 'VIEWS_TODAY', 'VIEWS_LAST_WEEK', 'DAYS',
    'SQUARE_SUM', 'SQUARE_LIVE', 'SQUAER_KITCHEN'
]

num_features_frequent = [
    'ROOMS', 'HEIGHT', 'BUILD_YEAR', 'REPAIR'
]

In [29]:
inp_mean = SimpleImputer(strategy='mean', missing_values=np.nan)
inp_max = SimpleImputer(strategy='constant', missing_values=np.nan, fill_value=10000)
inp_most_frequent = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
inp_dist = SimpleImputer(strategy='constant', missing_values=np.nan, fill_value='Other')

In [30]:
df[num_features_mean] = inp_mean.fit_transform(df[num_features_mean])
df[num_features_max] = inp_max.fit_transform(df[num_features_max])
df[num_features_frequent] = inp_most_frequent.fit_transform(df[num_features_frequent])
df[['DISTRICT']] = inp_dist.fit_transform(df[['DISTRICT']])

In [31]:
p = pd.get_dummies(df['AREA'], prefix='AREA')
df = pd.concat([df, p], axis=1)
df.drop(['AREA'], axis=1, inplace=True)

In [32]:
p = pd.get_dummies(df['DISTRICT'], prefix='DISTRICT')
df = pd.concat([df, p], axis=1)
df.drop(['DISTRICT'], axis=1, inplace=True)

In [33]:
df.drop(['TOILET', 'BALCONY'], axis=1, inplace=True)

In [34]:
df.drop(['FULL'], axis=1, inplace=True)

In [35]:
inp_coeff = SimpleImputer(strategy='max', missing_values=np.nan)

In [36]:
df

,COORD_X,COORD_Y,CITY,UNDERGROUND,ROOMS,HEIGHT,BUILD_YEAR,REPAIR,PRICE,TERMS_SALE,SHOP,PHARMACY,AGENCY,VIEWS_TODAY,VIEWS_LAST_WEEK,FLOOR_NOW,FLOOR_MAX,SQUARE_SUM,SQUARE_LIVE,SQUAER_KITCHEN,DAYS,LOC_TYPE,COEFF,AREA_Брестская область,AREA_Витебская область,AREA_Гомельская область,AREA_Гродненская область,AREA_Минская область,AREA_Могилевская область,DISTRICT_Other,DISTRICT_Заводской район,DISTRICT_Ленинский район,DISTRICT_Московский район,DISTRICT_Октябрьский район,DISTRICT_Партизанский район,DISTRICT_Первомайский район,DISTRICT_Советский район,DISTRICT_Сокол,DISTRICT_Фрунзенский район,DISTRICT_Центральный район
0,30.247803,53.927870,г. Гродно,10000.0,1.0,2.52,1980.0,0,21526.0,1,200.0,500.0,0,3.0,81.0,6.0,9.0,57.0,20.0,7.000000,60.000000,г.,0.000833,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
1,23.816521,53.697233,г. Гродно,10000.0,1.0,2.52,1980.0,0,21526.0,1,200.0,500.0,0,4.0,82.0,6.0,9.0,57.0,20.0,7.000000,60.000000,г.,0.000844,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
2,30.247803,53.927870,г. Кобрин,10000.0,47.0,2.50,1979.0,0,22551.0,1,200.0,200.0,1,6.0,69.0,5.0,5.0,49.1,14.0,7.200000,93.000000,г.,0.000457,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,24.353911,52.208929,г. Кобрин,10000.0,47.0,2.50,1979.0,0,22551.0,1,200.0,200.0,1,8.0,71.0,5.0,5.0,49.1,14.0,7.200000,93.000000,г.,0.000470,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,30.247803,53.927870,г. Марьина Горка,10000.0,1.0,2.50,1949.0,1,22551.0,1,350.0,700.0,1,12.0,114.0,2.0,4.0,71.4,48.1,9.300000,3.000000,г.,0.023506,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12781,27.698900,53.946600,г. Минск,800.0,8.0,3.00,2005,0,399770.0,1,50.0,200.0,1,36.0,67.0,5.0,6.0,175.0,108.0,13.100000,2.000000,г.,0.020722,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
12782,27.391663,53.915300,д. Тарасово,10000.0,8.0,2.64,2012,3,861042.0,1,10000.0,10000.0,1,3.0,33.0,2.0,NaN,254.7,151.8,61.300000,2.000000,д.,0.010205,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
12783,27.677200,53.946800,г. Минск,700.0,8.0,2.50,2000,3,1170000.0,1,100.0,200.0,1,12.0,95.0,7.0,9.0,304.3,211.3,19.800000,2.000000,г.,0.029383,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
12784,27.678500,53.883600,г. Минск,10000.0,9.0,3.20,2010,1,389519.0,1,300.0,200.0,1,15.0,61.0,3.0,NaN,307.4,191.3,9.568542,59.000000,г.,0.000638,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0


In [37]:
def height_prep(height):
    if not (0 < height < 5) and not pd.isna(height):
        if height > 1000:
            return height / 1000
        elif height < 0:
            return np.abs(height)
    elif pd.isna(height):
        return height    
    return float(height)
    
df['HEIGHT'] = df['HEIGHT'].map(height_prep)

In [40]:
df.drop(['CITY'], inplace=True, axis=1)

In [41]:
from sklearn.preprocessing import LabelEncoder

In [47]:
inp_dist = SimpleImputer(strategy='constant', missing_values=None, fill_value='Other')

In [48]:
df[['LOC_TYPE']] = inp_dist.fit_transform(df[['LOC_TYPE']])

In [50]:
le = LabelEncoder()
df[['LOC_TYPE']] = le.fit_transform(df[['LOC_TYPE']])

In [52]:
df.to_csv('realt_prep.csv', index=False)